## Importamos librerias a usar

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

## Comenzamos la extraccion

In [2]:

COUNT_IMAGES=400

def save_data(input_src):
        modelo_manos=mp.solutions.hands
        hands= modelo_manos.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.9
        ) 
        X=[]
        Y=[]
        for video_ruta in os.listdir(input_src): 
                video=cv2.VideoCapture(f"{input_src}/{video_ruta}")
                fps = video.get(cv2.CAP_PROP_FPS)
                data=[video_ruta.split(".")[0]]*COUNT_IMAGES
                Y.extend(data)
                print(Y[-1])
                index=0
                while index<COUNT_IMAGES:
                    r,image=video.read()
                    if r is None or image is None:
                            continue
                    height, width, _ = image.shape
                    new_width = int(width / 2)
                    new_height = int(height / 2)
                    image = cv2.resize(image, (new_width, new_height))
                    image=cv2.flip(image,1) 
                    image_rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                    results=hands.process(image_rgb)
                    if results.multi_hand_landmarks is not None:
                        for marcador_manos in  results.multi_hand_landmarks:
                                landmarks=[]
                                for punto in marcador_manos.landmark:
                                        landmarks=landmarks+ [punto.x, punto.y]
                                if index>=COUNT_IMAGES:
                                        break
                                X.append(landmarks)
                                index+=1

                    if  (cv2.waitKey(1)  & 0xFF == ord('q')):
                                    break
                    image=cv2.flip(image,1)
                video.release()
                cv2.destroyAllWindows()
        print(np.array(X).shape)
        print(np.array(Y).shape)
        print(np.unique(Y))
        return np.array(X),np.array(Y,dtype='O')


In [ ]:
X,Y=save_data("./videos")

1


c:\Users\laime\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


A
B
BOLIVIA


In [ ]:
Y

array(['1', '1', '1', ..., '_', '_', '_'], dtype=object)

## Guardamos los datos cargados

Guardamos las 23 letras del abecedario 
de cada letra usamos 400 fotos. De cada foto son 21 puntos de la mano y de cada punto sacamos las coordenadas x,y

In [ ]:
import h5py
with h5py.File("datos.h5", 'w') as hf:
        hf.create_dataset('X', data=X)
        hf.create_dataset('Y', data=Y)